<a href="https://colab.research.google.com/github/Murcha1990/ML_AI24/blob/main/Hometasks/Pro/AI_HW1_Regression_with_inference_pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Домашнее задание №1 (pro)**

В этом домашнем задании вам будет необходимо:
*  обучить модель регрессии для предсказания стоимости автомобилей;
* реализовать веб-сервис для применения построенной модели на новых данных

> Оценка за домашку = $min(\text{ваш балл}, 11)$

**Мягкий дедлайн: 27 ноября 23:59**

**Жесткий дедлайн: 20 декабря 23:59 (конец модуля)**



**Примечание**

В каждой части оцениваются как код, **так и ответы на вопросы.** Вопросы подсвечены синим цветом.

Если нет одного и/или другого, то часть баллов за соответствующее задание снимается.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
from ydata_profiling import ProfileReport
import re
from scipy.stats import spearmanr, kendalltau
import phik
from phik import resources
from phik.report import plot_correlation_matrix


random.seed(42)
np.random.seed(42)
pd.options.display.float_format = '{:.2f}'.format

### **Задание 0**
Для чего фиксируем сиды в домашках?

Для воспроизводимости результатов

# **Часть 1 | EDA**


Первая часть состоит из классических шагов EDA:

- Базовый EDA и обработка признаков (1.7 балла)
- Визуализации признаков и их анализ (1.6 балл)

Всего можно набрать 3.3 основных балла и 0.25 бонусных.

В следующих частях, вы увидите бонусные задания. Бонусные задания выделены как **Дополнительное задание/Бонус**. Вы можете выполнять их, чтобы в случае ошибок в основных задачах всё равно набрать за работу максимум. Кроме того, дополнительные задания позволяют вам углубить знания.

Призываем активно использовать их!

## **Простейший EDA и обработка признаков (1.7 балла)**

In [2]:
df_train = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_train.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_test.csv')
df = pd.concat([df_train, df_test]).reset_index(drop=True)
print("Train data shape:", df_train.shape)
print("Test data shape: ", df_test.shape)

Train data shape: (6999, 13)
Test data shape:  (1000, 13)


### **Задание 1.(0.4 балла)**
Выполните операции, направленные на базовое исследование данных:

- [ ] Посмотрите, есть ли в датасете пропуски и дубликаты. Запишите/выведите названия колонок, для которых есть пропущенные значения (0.1 балла)
- [ ] Посмотрите, есть ли в данных явные дубликаты (0.05 балла)
- [ ] Постройте дашборд в одну строку, используя [ydata-profilling](https://github.com/ydataai/ydata-profiling)(0.15 балла)
- [ ] Опишите базовые выводы (какие — выберите сами), используя дашборд (0.2 балла)

In [3]:
columns_with_missing_values = [
    column for column in df.columns
    if df[column].isna().any() or (df[column] == ' ').any()
]
print(f'В ДС есть пропуски в столбцах: {", ".join(columns_with_missing_values)},\nВ ДС есть явные дубликаты: {not df[df.duplicated(keep=False)].empty}')

В ДС есть пропуски в столбцах: mileage, engine, max_power, torque, seats,
В ДС есть явные дубликаты: True


In [4]:
pr = ProfileReport(df)

In [5]:
pr.to_file('cars_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Наибольшая корреляция, у selling price с year и transmission. Есть пропуски в 5 признаках.
mileage	engine	max_power и torque нуждаются в преобразовании. name нужно переделывать, так как большое кол-во различных объектов. Распределения числовых признаков выглядят не совсем стандартно.

### **Задание 2 (0.2 балла)**
Проанализируйте статистики датасета.

**Ваша задача:**
- [ ] Посчитайте основные статистики по числовым столбцам для трейна и теста
- [ ] Посчитайте основные статистики по категориальным столбцам для трейна и теста
- [ ] Сравните среднее и медиану внутри `train`, внутри `test` и между собой. О чём могут говорить результаты?

**Подсказка:**

Используте ``.describe()`` с нужным(и) аргументом(-ами).

In [6]:
df_train.describe(include='number')

,year,selling_price,km_driven,seats
count,6999.00,6999.00,6999.00,6797.00
mean,2013.82,639515.20,69584.62,5.42
std,4.05,808941.91,57724.00,0.97
min,1983.00,29999.00,1.00,2.00
25%,2011.00,254999.00,35000.00,5.00
50%,2015.00,450000.00,60000.00,5.00
75%,2017.00,675000.00,97000.00,5.00
max,2020.00,10000000.00,2360457.00,14.00


In [7]:
df_test.describe(include='number')

,year,selling_price,km_driven,seats
count,1000.00,1000.00,1000.00,981.00
mean,2013.68,617901.04,71393.34,5.41
std,4.01,758553.86,48486.22,0.92
min,1995.00,31000.00,1303.00,4.00
25%,2011.00,250000.00,37000.00,5.00
50%,2014.00,434999.00,61500.00,5.00
75%,2017.00,670000.00,100000.00,5.00
max,2020.00,6000000.00,375000.00,9.00


In [8]:
df_train.describe(exclude='number')

,name,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque
count,6999,6999,6999,6999,6999,6797,6797,6803,6796
unique,1924,4,3,2,5,386,120,316,419
top,Maruti Swift Dzire VDI,Diesel,Individual,Manual,First Owner,18.9 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm
freq,116,3793,5826,6095,4587,197,885,330,468


In [9]:
df_test.describe(exclude='number')

,name,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque
count,1000,1000,1000,1000,1000,981,981,981,981
unique,621,4,3,2,5,237,88,182,226
top,Maruti Alto 800 LXI,Diesel,Individual,Manual,First Owner,18.6 kmpl,1248 CC,74 bhp,200Nm@ 1750rpm
freq,15,534,837,877,623,23,116,43,57


Стандартное отклонение относительно сильно различается на тесте у km_driven, относительно трейна
У среднего арифм на трейне и тесте близкие значения
У медианы на трейне и тесте близкие значения

### **Задание 3 (0.2 балла)**

- [ ] Посмотрите, есть ли в трейне объекты с одинаковым признаковым описанием (целевую переменную следует исключить). Если есть, то сколько? (0.05 балла)
- [ ] Отобразите такие объекты (0.05 балла)
- [ ] Удалите повторяющиеся строки. Если при одинаковом признаковом описании цены на автомобили отличаются, то оставьте первую строку по этому автомобилю (0.05 балла)
- [ ]  Обновите индексы строк таким образом, чтобы они шли от 0 без пропусков (0.05 балла)

In [10]:
print(df_train[df_train.duplicated(subset=df_train.columns.difference(['selling_price']), keep=False)].shape[0])

df_train[df_train.duplicated(subset=df_train.columns.difference(['selling_price']), keep=False)]

1799


,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
22,Maruti Baleno Delta 1.2,2016,575000,45000,Petrol,Individual,Manual,First Owner,21.4 kmpl,1197 CC,83.1 bhp,115Nm@ 4000rpm,5.00
26,Maruti Omni E MPI STD BS IV,2018,254999,25000,Petrol,Individual,Manual,First Owner,16.8 kmpl,796 CC,34.2 bhp,59Nm@ 2500rpm,8.00
27,Maruti Vitara Brezza LDi Option,2017,670000,70000,Diesel,Individual,Manual,First Owner,24.3 kmpl,1248 CC,88.5 bhp,200Nm@ 1750rpm,5.00
28,Fiat Palio 1.2 ELX,2003,70000,50000,Petrol,Individual,Manual,Second Owner,NaN,NaN,NaN,NaN,NaN
29,Maruti Omni 8 Seater BSIV,2012,150000,35000,Petrol,Individual,Manual,Second Owner,14.0 kmpl,796 CC,35 bhp,6.1kgm@ 3000rpm,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6981,Ford Freestyle Titanium Plus Diesel BSIV,2018,750000,24000,Diesel,Individual,Manual,First Owner,24.4 kmpl,1498 CC,98.63 bhp,215Nm@ 1750-3000rpm,5.00
6984,Ford Freestyle Titanium Plus Diesel BSIV,2018,746000,24000,Diesel,Individual,Manual,First Owner,24.4 kmpl,1498 CC,98.63 bhp,215Nm@ 1750-3000rpm,5.00
6989,Maruti Swift Dzire VDI,2015,625000,50000,Diesel,Individual,Manual,First Owner,26.59 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.00
6997,Tata Indigo CR4,2013,290000,25000,Diesel,Individual,Manual,First Owner,23.57 kmpl,1396 CC,70 bhp,140Nm@ 1800-3000rpm,5.00


In [11]:
df_train = df_train.drop_duplicates(subset=df_train.columns.difference(['selling_price']), keep='first').reset_index(drop=True)

### **Задание 4 (0.3 балла)**

Вы могли заметить, что с признаками ``mileage, engine, max_power и torque`` всё не очень хорошо. Они распознаются как строки (можно убедиться в этом, вызвав `data.dtypes`). Однако эти переменные не являются категориальными — они — числа. Соответственно, нужно привести их к числовому виду.

**Задача :**
* [ ] Уберите единицы измерения для признаков ``mileage, engine, max_power``.
* [ ] Приведите тип данных к ``float``.
* [ ] Предобработайте признак `torque` — разделите его на два: собственно `torque` и `max_torque_rpm`. Учтите единицы измерения


**Важно**
- Все действия нужно производить над обоими датасетами — `train` и `test`.

In [12]:
for frame in [df_train, df_test]:
    for col in ['mileage', 'engine', 'max_power']:
        frame[col] = np.where(
            frame[col].notna(),
            frame[col].str.replace(r'[^0-9\.]', '', regex=True).replace('', 0).astype('float'),
            frame[col]
        )
        frame[col] = frame[col].astype('float')

In [13]:
def process_torque(row):
    row = row.strip().lower()
    
    match_nm = re.search(r'(\d+\.?\d*)\s*nm@?\s*(\d{1,5}(?:[-~]\d{1,5})?)\s*rpm', row)
    if match_nm:
        torque = float(match_nm.group(1))
        rpm_range = match_nm.group(2)
        if '-' in rpm_range :
            rpm = int(rpm_range.split('-')[0].replace(',', ''))
        else:
            rpm = int(rpm_range.split('~')[0].replace(',', ''))
        return torque, rpm
    
    match_kgm = re.search(r'(\d+\.?\d*)\s*kgm(?:\s*at\s*)?(\d{1,5}(?:[-~]\d{1,5})?)\s*rpm', row)
    if match_kgm:
        torque_kgm = float(match_kgm.group(1))
        torque = torque_kgm * 9.8
        rpm_range = match_kgm.group(2)
        if '-' in rpm_range :
            rpm = int(rpm_range.split('-')[0].replace(',', ''))
        else:
            rpm = int(rpm_range.split('~')[0].replace(',', ''))
        return torque, rpm
    
    match_kgm_at = re.search(r'(\d+\.?\d*)\s*kgm@\s*(\d{1,5}(?:[-~]\d{1,5})?)\s*rpm', row)
    if match_kgm_at:
        torque_kgm = float(match_kgm_at.group(1))
        torque = torque_kgm * 9.80665
        rpm_range = match_kgm_at.group(2)
        rpm = int(rpm_range.split('-')[0].replace(',', ''))
        return torque, rpm
    
    match_generic = re.search(r'(\d+\.?\d*)\s*@\s*(\d{1,5}(?:[-~]\d{1,5})?)\s*\(kgm@\s*rpm\)', row)
    if match_generic:
        torque_generic = float(match_generic.group(1))
        torque = torque_generic * 9.8
        rpm_range = match_generic.group(2)
        rpm = int(rpm_range.split('-')[0].replace(',', ''))
        return torque, rpm
    return None, None 


df_train[['torque', 'max_torque_rpm']] = df_train['torque'].astype('str').apply(lambda x: pd.Series(process_torque(x)))
df_test[['torque', 'max_torque_rpm']] = df_test['torque'].astype('str').apply(lambda x: pd.Series(process_torque(x)))


На первом шаге мы обнаружили пропуски. Давайте избавимся от них.

**Задание:**
- [ ] Заполните пропуски в столбцах медианами. Убедитесь, что после заполнения пропусков не осталось.
- [ ] Почему стоит применять именно медиану. Могли ли мы применить среднее? Обоснуйте свое рассуждение.
- [ ] Как правильно считать медиану для заполнения? Выберите верное утверждение:
 - По тестовым свою, по тренировочным — свою
 - По тренировочным данным для `train` и `test`






In [14]:
for col in df_test.select_dtypes(include='number').columns:
    median = df_test[col].median()
    df_train[col].fillna(median, inplace=True)
    df_test[col].fillna(median, inplace=True)

In [15]:
pd.concat([df_train, df_test]).info()

<class 'pandas.core.frame.DataFrame'>
Index: 6840 entries, 0 to 999
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            6840 non-null   object 
 1   year            6840 non-null   int64  
 2   selling_price   6840 non-null   int64  
 3   km_driven       6840 non-null   int64  
 4   fuel            6840 non-null   object 
 5   seller_type     6840 non-null   object 
 6   transmission    6840 non-null   object 
 7   owner           6840 non-null   object 
 8   mileage         6840 non-null   float64
 9   engine          6840 non-null   float64
 10  max_power       6840 non-null   float64
 11  torque          6840 non-null   float64
 12  seats           6840 non-null   float64
 13  max_torque_rpm  6840 non-null   float64
dtypes: float64(6), int64(3), object(5)
memory usage: 801.6+ KB


Среднее арифметическое искажается, если в данных есть выброс, тогда как медиана, как центральное значение в упорядоченном наборе, устойчива к выбросам и лучше отражает типичное значение данных. Если в конкретном признаке у нас было бы симметричное распрееление, то могли бы заполнить средним (арифметическим)
Что бы не "Подглядывать в тест" правильно считать на трейне и заполнять для теста и трейна (2 верно)

### **Задание 6 (0.2 балла)**

Теперь, когда не осталось пропусков, давайте преобразуем столбцы к более подходящим типам. А именно столбцы ``engnine`` и ``seats`` к приведем к `int`.

- [ ] Осуществите приведение столбцов к необходимому типу.
- [ ] Ответье на вопрос — почему (хоть мы этого и не делаем) ``seats``, возможно рассмотреть как категориальную переменную?

In [16]:
df_test['engine'] = df_test['engine'].astype('int')
df_test['seats'] = df_test['seats'].astype('int')

df_train['engine'] = df_train['engine'].astype('int')
df_train['seats'] = df_train['seats'].astype('int')

Так как по факту кол-во мест в машине это фиксированная группа (категория), кроме того нет упорядоченности и линейной взаимосвязи

### **Задание 7 (0.1 балла)**

Снова вызовите метод describe и проанализируйте статистики.

**Ответье на вопрос:**
- [ ] Есть ли основания предполагать, что заполнение пропусков свдинуло наши распределения? Могло ли это вообще возникнуть?

In [17]:
df_train.describe(include='number')

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,max_torque_rpm
count,5840.00,5840.00,5840.00,5840.00,5840.00,5840.00,5840.00,5840.00,5840.00
mean,2013.43,522960.09,73952.24,19.43,1429.45,87.93,173.34,5.43,2606.60
std,4.10,535432.00,60071.14,3.99,485.66,31.65,86.53,0.98,1064.94
min,1983.00,29999.00,1.00,0.00,624.00,0.00,47.07,2.00,175.00
25%,2011.00,250000.00,39000.00,16.95,1197.00,68.00,113.00,5.00,1750.00
50%,2014.00,405000.00,70000.00,19.33,1248.00,81.86,172.50,5.00,2000.00
75%,2017.00,640000.00,100000.00,22.30,1498.00,99.00,200.00,5.00,3500.00
max,2020.00,10000000.00,2360457.00,42.00,3604.00,400.00,1274.00,14.00,5000.00


In [18]:
# assert df_train.shape == (5840, 13)

In [19]:
df_test.describe(include='number')

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,max_torque_rpm
count,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00
mean,2013.68,617901.04,71393.34,19.34,1454.88,90.87,176.59,5.40,2642.44
std,4.01,758553.86,48486.22,3.95,522.00,34.89,92.58,0.91,1090.36
min,1995.00,31000.00,1303.00,0.00,624.00,34.20,48.00,4.00,1000.00
25%,2011.00,250000.00,37000.00,16.55,1197.00,69.00,113.00,5.00,1750.00
50%,2014.00,434999.00,61500.00,19.33,1248.00,83.10,172.50,5.00,2000.00
75%,2017.00,670000.00,100000.00,22.30,1582.00,102.00,202.00,5.00,3625.00
max,2020.00,6000000.00,375000.00,32.26,3604.00,280.00,1274.00,9.00,5000.00


В результате заполнение медианами данные при большом кол-ве пропусков могут сконцентрироваться вокруг медианного значения, приводя к уменьшению дисперсии и уменьшению хвостов распределения. Сдвигать распределение в сторону не должно.
В целом распределения не изменились, однако изменилось значение среднего арифм и медианы на трейну у цены продажи, однако скорее всего, это из-за удаления дубликотов

## **Визуализации и корреляция (1.6 балла + 0.25)**

Визуализация данных — важный шаг в работе. Визуализировать данные необходимо, например, чтобы:

- Оценить распределения признаков самих по себе (это может натоклнуть вас на мысли о модели, которую можно использовать)
- Сравнить распределения на `train` и `test` — чтобы проверить, насколько информация, на которой вы будете обучаться согласуется с той, на которой модель должна работать
- Оценить есть ли явная связь признаков с целевой переменной

**Важно:**

Если распределения на `train` и `test` не совпадают, это не значит, что нужно перемешивать данные! Более корректно актуализировать задачу и уточнить, а не устарели ли данные `train`. Также полезным может быть собрать новую тестовую выборку, смешав те, что имеются сейчас.

**Если вы будете подгонять распределения, то можете встретиться с переобучением!**

### **Задание 8 (0.5 балла)**

Шаг 1.
- [ ] Воспользуйтесь `pairplot` из библиотеки `seabron`, чтобы визуализировать попарные распределения числовых признаков для `train`
- [ ] По полученному графику ответьте на вопросы:
 - Можно ли предположить на основе распределений связь признаков с целевой переменной?
 - Можно ли предположить на основе распределений выдвинуть гипотезу о корреляциях признаков?

Шаг 2.

- [ ] Постройте pairplot по тестовым данным
- [ ] Ответьте на вопрос "Похожими ли оказались совокупности при разделении на трейн и тест?"

Шаг 1. Я бы предположил наличие взаимосвязи между признаками yea, engine и целевой переменной. Предположение о связи можно выдвигать, так как по облаку распределения прослеживается закономерность.
Можно, например можно проследить наличие положительной линейной корреляции между признаками torque и engine

In [20]:
plt.figure(figsize=(15, 10))
sns.pairplot(df_train)

In [21]:
plt.figure(figsize=(15, 10))
sns.pairplot(df_test)

Шаг 2. 
Совокупности в целом оказались похожими, после разделения не трейн и тест

### **Задание 9 (0.5 балла)**

И так, вы выдвинули гипотезы о наличии связи. Теперь давайте оценим эту связь в числах.

**Задание:**
- [ ] Получите значения коэффициента корреляции Пирсона для тренировочного набора данных при помощи `pd.corr()`
- [ ] По полученным корреляциям постройте тепловую карту (`heatmap` из бибилотеки seaborn)

In [22]:
sns.heatmap(df_train.select_dtypes(include='number').corr())

<Axes: >

- [ ] Ответьте на вопросы:
 - Какие 2 признака наименее скоррелированы между собой?
 - Между какими наблюдается довольно сильная положительная линейная зависимость?
 - Правильно ли, опираясь на данные, утверждать, что чем меньше год, тем, скорее всего, больше километров проехала машина к дате продажи?

In [23]:
df_train.select_dtypes(include='number').corr()

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,max_torque_rpm
year,1.00,0.43,-0.37,0.34,0.00,0.16,0.09,0.04,0.07
selling_price,0.43,1.00,-0.16,-0.10,0.45,0.69,0.57,0.15,-0.22
km_driven,-0.37,-0.16,1.00,-0.18,0.23,0.02,0.12,0.19,-0.26
mileage,0.34,-0.10,-0.18,1.00,-0.57,-0.37,-0.24,-0.45,0.02
engine,0.00,0.45,0.23,-0.57,1.00,0.68,0.69,0.65,-0.48
max_power,0.16,0.69,0.02,-0.37,0.68,1.00,0.77,0.24,-0.27
torque,0.09,0.57,0.12,-0.24,0.69,0.77,1.00,0.34,-0.62
seats,0.04,0.15,0.19,-0.45,0.65,0.24,0.34,1.00,-0.34
max_torque_rpm,0.07,-0.22,-0.26,0.02,-0.48,-0.27,-0.62,-0.34,1.00


Наименьшая: km_driven и max_power, engine и year
Сильная положительная: max_power и torque, max_power и engine
Да можно так утверждать, но не очень уверенно

### **Задание 10 (0.6 балла)**

По умолчанию `pd.corr` возвращает корреляцию Пирсона, говорящую о линейной взаимосвязи. Но зависимости существуют не только линейные! В этой задаче, попробуйте измерить другие способы вычисления корреляций.

- [ ] Реализуйте корреляцию Спирмена/Кендала (на выбор) без использования библиотек (можно пользоваться только `numpy`). Сравните результаты вычисления с библиотечной реализацией
- [ ]

Постройте матрицу корреляции [phik](https://pypi.org/project/phik/)
- [ ] Проинтерпретируйте результаты


In [24]:
def spearman_corr_manual(x, y):
    rank_x = np.argsort(np.argsort(x))
    rank_y = np.argsort(np.argsort(y))
    
    n = len(x)
    
    diff_rank = rank_x - rank_y
    numerator = 6 * np.sum(diff_rank ** 2)
    denominator = n * (n**2 - 1)
    
    spearman_corr = 1 - (numerator / denominator)
    return spearman_corr

spearman_corr = spearman_corr_manual(df_train['year'], df_train['selling_price'])
spearman_corr_scipy, _ = spearmanr(df_train['year'], df_train['selling_price'])
print(spearman_corr, spearman_corr_scipy)

0.7050604986492461 0.707122567874714


In [25]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5840 entries, 0 to 5839
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            5840 non-null   object 
 1   year            5840 non-null   int64  
 2   selling_price   5840 non-null   int64  
 3   km_driven       5840 non-null   int64  
 4   fuel            5840 non-null   object 
 5   seller_type     5840 non-null   object 
 6   transmission    5840 non-null   object 
 7   owner           5840 non-null   object 
 8   mileage         5840 non-null   float64
 9   engine          5840 non-null   int32  
 10  max_power       5840 non-null   float64
 11  torque          5840 non-null   float64
 12  seats           5840 non-null   int32  
 13  max_torque_rpm  5840 non-null   float64
dtypes: float64(4), int32(2), int64(3), object(5)
memory usage: 593.3+ KB


In [26]:
phik_matrix = df_train.phik_matrix(interval_cols=['year', 'selling_price', 'km_driven', 'mileage', 'engine', 'max_power', 'torque', 'seats', 'max_torque_rpm'])

plt.figure(figsize=(10, 8))
plot_correlation_matrix(phik_matrix.values, 
                        x_labels=phik_matrix.columns, 
                        y_labels=phik_matrix.index, 
                        vmin=0, vmax=1, title="Phik Correlation Matrix")

C:\Users\vyacheslav\anaconda3\envs\job1\Lib\site-packages\phik\data_quality.py:59: UserWarning: The number of unique values of variable name is large: 1924. Are you sure this is not an interval variable? Analysis for pairs of variables including name can be slow.
  warnings.warn(


Целевая переменная больше всего коррелирует с max_power, mileage и name. Ожидается неплохая модель. Мультиколлинеарности не обнаружено

### **Дополнительные визуализации (бонус 0.2 балла)**

Если вам кажется, что мы не попросили вас нарисовать какие-то очень важные зависимости, нарисуйте их и поясните.

In [27]:
import matplotlib.pyplot as plt
import numpy as np


plt.figure(figsize=(8, 6))
plt.boxplot(df_train.select_dtypes(include='number'), vert=True, patch_artist=True, labels=df_train.select_dtypes(include='number').columns)
plt.yscale('log')

# plt.ylabel('Values')
plt.xlabel('Groups')
plt.grid(axis='y', linestyle='--', alpha=0.7)

C:\Users\vyacheslav\AppData\Local\Temp\ipykernel_10796\473355673.py:6: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(df_train.select_dtypes(include='number'), vert=True, patch_artist=True, labels=df_train.select_dtypes(include='number').columns)


Признаки имеют сильно разные масштабы -> нормализовать и стандартизировать данные перед использованием,
Многие признаки имеют выраженные выбросы. Возможно, стоит рассмотреть методы обработки выбросов. Кроме того, заметим, что признаки selling_price и km_driven варьируются в очень широком диапазоне значений.

# **Часть 2 (1.7 балла) | Модель только на вещественных признаках**

В этой части вам предстоит обучить модель только на вещественных признаках. Почему только на них?

Чем больше признаковое пространство — чем сложнее модель. А чем модель проще — тем лучше для скорости работы и интерпретации признаков.

За задания этой части вы можете набрать 1.7 основных и 0.15 бонусных балла;

### **Задание 11 (0.05 балла)**

Разбейте данные на тренировочный и тестовый наборы. Перед разбиением создайте копию датафрейма, который будет хранить только вещественные признаки и используйте его (то есть категориальные столбцы (все, кроме seats) необходимо удалить).

В переменные y_train и y_test запишите значения целевых переменных.

In [28]:
y_train = df_train[['selling_price']]
X_train = df_train.drop(columns=['selling_price', 'name', 'fuel', 'seller_type', 'transmission', 'owner'])

In [29]:
X_train.columns # Расхождение из-за разбиения столбца torque

Index(['year', 'km_driven', 'mileage', 'engine', 'max_power', 'torque',
       'seats', 'max_torque_rpm'],
      dtype='object')

In [30]:
# assert X_train.shape == (5840, 6)

In [31]:
y_test = df_test[['selling_price']]
X_test = df_test.drop(columns=['selling_price', 'name', 'fuel', 'seller_type', 'transmission', 'owner'])

In [32]:
# assert X_test.shape == (1000, 6)

### **Задание 12. (0.2 балла)**

Построим нашу первую модель!
- [ ] Обучите классическую линейную регрессию с дефолтными параметрами. Посчтитайте $R^2$ и $MSE$ для трейна и для теста.
- [ ] Сделайте выводы по значениям метрик качества.

**Примечание:**

Здесь и далее $R^2$ и $MSE$ для трейна и для теста выводите везде, где требуется обучать модели, даже если в явном виде этого не просят. Иначе непонятно, как понять, насколько успешны наши эксперименты.

In [33]:
def summary(sk_model, ytest, ytrain, xtest, xtrain):
    train_pred = sk_model.predict(xtrain)
    test_pred = sk_model.predict(xtest)
    print(f'На трейне R2: {r2_score(ytrain, train_pred)}, MSE: {MSE(ytrain, train_pred)}')
    print(f'На тесте R2: {r2_score(ytest, test_pred)}, MSE: {MSE(ytest, test_pred)}')

In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error as MSE

model = LinearRegression()

model.fit(X_train, y_train)

summary(model, y_test, y_train, X_test, X_train)

На трейне R2: 0.5985602496613212, MSE: 115068022987.88373
На тесте R2: 0.5941939085731405, MSE: 233268929404.18417


MSE увеличился на тестовой выборке, потому что модель ошибается сильнее на новых данных, однако она продолжает объяснять около 60% вариации данных как на обучающей, так и на тестовой выборке.

### **Задание 13 (0.15 балла)**

- [ ] Реализуйте $R^2$ руками. Приведите формулу $R^2$ и объясните каждую компоненту метрики

In [35]:
def r2_score(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    y_mean = np.mean(y_true)
    
    rss = np.sum((y_true - y_pred) ** 2)
    
    tss = np.sum((y_true - y_mean) ** 2)
    
    r2 = 1 - (rss / tss)
    return r2
r2_score(y_test, model.predict(X_test))

0.5941939085731405

RSS - сумма квадрата ошибок модели. т.е. показывает насколько факт отклоняется от предсказания
TSS - сумма квадратов отклонений от среднего значения, т.е. измеряет общую вариацию зависимой переменной


### **Бонус (0.15 балла)**

- [ ] Реализуйте [$\text{adjusted}-R^2$](https://en.wikipedia.org/wiki/Coefficient_of_determination).
- [ ] Объясните, когда применяется $\text{adjusted}-R^2$?

In [36]:
def adjusted_r2(y_true, y_pred, n, k):
    r2 = r2_score(y_true, y_pred)
    return 1 - (1 - r2) * (n - 1) / (n - k - 1)


Используется для оценки качества модели регрессии, учитывая влияние числа x. Например, если R2 увеличивается, но Adjusted R2 уменьшается, это говорит о том, что новые x ухудшают обобщающую способность модели.

### **Задание 14 (0.05 балла)**

Всегда есть место совершенству. Поэтому давайте попробуем улучшить модель. При помощи стандартизации признаков.

- [ ] Стандартизируйте значения в тренировочных и тестовых данных. Стандартизатор **обучайте только на `train`**.

In [37]:
from sklearn.preprocessing import StandardScaler

normalizer = StandardScaler()
X_train_norm = normalizer.fit_transform(X_train)
X_test_norm = normalizer.transform(X_test)

model.fit(X_train_norm, y_train)

summary(model, y_test, y_train, X_test_norm, X_train_norm)

На трейне R2: 0.5985602496613207, MSE: 115068022987.88383
На тесте R2: 0.5941939085731455, MSE: 233268929404.18127


### **Задание 15 (0.1 балла)**

Хотя стандартизация не помогла сильно прибавить в качестве она открыла возможность интерпретировать важность признаков в модели. Правило интерпретации такое:

Чем больше коэффициент $\beta_i$ по модулю, тем важнее признак.

**Ответьте на вопрос:**

- [ ] Какой признак оказался наиболее информативным в предсказании цены?

In [38]:
model.coef_

array([[165362.92846273, -48870.53295592,  -1546.39494169,
         21776.59359082, 301147.84030456,  33077.30654872,
        -30048.42062051, -40840.59068633]])

max_power самый информативный

### **Задание 16 (0.25 балла)**

Попробуем улучшить нашу модель с помощью применения регуляризации. Для этого воспльзуемся `Lasso` регрессией.  Кроме того, попробуйте использовать её теоретическое свойство отбора признаков, за счет зануления незначимых коэффициентов.

**Задание:**

- [ ] Обучите Lasso регрессию на тренировочном наборе данных с нормализованными признаками. Оцените её качество
- [ ] Проверьте, занулила ли L1-регуляризация с параметрами по умолчанию какие-нибудь веса? Предположите почему.

In [39]:
from sklearn.linear_model import Lasso

lasso_reg = Lasso()
lasso_reg.fit(X_train_norm, y_train)
summary(lasso_reg, y_test, y_train, X_test_norm, X_train_norm)
lasso_reg.coef_

На трейне R2: -9334.565424040831, MSE: 115068023000.86218
На тесте R2: -1357.8843210898228, MSE: 233269626364.99963


array([165360.66951064, -48869.60473206,  -1543.28549663,  21774.67384377,
       301149.09209042,  33077.5327053 , -30044.46477529, -40838.36549349])

Модель не занулила никакой из параметров, причина в стандартном параметре регуляризации alpha, возможно его следует увеличить

### **Задание 17. Финальный рывок (0.4 балла)**

До этого мы с вами использовали `train` для обучения и `test` для прогнозирования. Но у нас есть ещё одна задача — подобрать оптимальные параметры модели. Для этого используем кросс-валидацию, описанную на семинарах.

Кроме того, выжмем максимум из модификаций регрессии. Построим `ElasticNet`. И сделаем всё по порядку.

**Ваша задача 1:**

- [ ] Перебором по сетке (c 10-ю фолдами) подберите оптимальные параметры для Lasso-регрессии. Вам пригодится класс [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).
- [ ] Ответьте на вопросы:
 - Сколько грид-сёрчу пришлось обучать моделей?
 - Что значит каждый параметр `param_grid`, который вы заполняли для `GridSearch`? Опишите каждый
 - Какой коэффициент регуляризации у лучшей из перебранных моделей? Занулились ли какие-нибудь из весов при такой регуляризации?

Модель подобрала параметры достаточно быстро, около 10 секунд

In [40]:
from sklearn.model_selection import GridSearchCV


param_grid = {'alpha': [0.01, 0.1, 0.5, 1, 5]}

lasso_grid_search = GridSearchCV(
    estimator=Lasso(), # Модель
    param_grid=param_grid, # Словарь гиперпараметров для перебора
    scoring='neg_mean_squared_error', # Метрика для оценки качества модели
    cv=10, # Кол-во флотов
    verbose=1, # Информация, которая будет выведена при итерациях
    n_jobs = -1  # Кол-во потоков параллельного выплнения 
)

lasso_grid_search.fit(X_train_norm, y_train)

Fitting 10 folds for each of 5 candidates, totalling 50 fits


GridSearchCV(cv=10, estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.1, 0.5, 1, 5]},
             scoring='neg_mean_squared_error', verbose=1)

In [41]:
lasso = Lasso(**lasso_grid_search.best_params_)
lasso.fit(X_train_norm, y_train)

summary(lasso, y_test, y_train, X_test_norm, X_train_norm)
lasso.coef_

На трейне R2: -9334.460271333888, MSE: 115068023311.79527
На тесте R2: -1357.8743027618448, MSE: 233272393321.2018


array([165351.60395444, -48865.9100109 ,  -1530.75078958,  21767.19449796,
       301154.13208453,  33078.34540578, -30028.69418152, -40829.43930726])

**Ваша задача 2:**

- [ ] Перебором по сетке (c 10-ю фолдами) подберите оптимальные параметры для [ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) регрессии.
- [ ] Ответьте на вопрос:
 - Сколько грид-сёрчу пришлось обучать моделей?
 - Какие гиперпараметры соответствуют лучшей (по выбранной метрике качества) из перебранных моделей?

Модель подобрала параметры достаточно быстро, около 2 секунд

In [42]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error as MSE

param_grid = {
    'alpha': [0.01, 0.1, 0.5, 1, 5],     # Сила регуляризации
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]  # Доля L1 регуляризации
}

elastic_net = ElasticNet()

elastic_net_grid_search = GridSearchCV(
    estimator=elastic_net, # Модель
    param_grid=param_grid,  # Словарь гиперпараметров для перебора
    scoring='neg_mean_squared_error',  # Используем MSE для оценки
    cv=10,  # 10-кратная кросс-валидация
    verbose=1,  # Вывод информации о ходе перебора
    n_jobs = -1  # Кол-во потоков параллельного выплнения 

)

elastic_net_grid_search.fit(X_train_norm, y_train)

Fitting 10 folds for each of 25 candidates, totalling 250 fits


GridSearchCV(cv=10, estimator=ElasticNet(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.1, 0.5, 1, 5],
                         'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]},
             scoring='neg_mean_squared_error', verbose=1)

In [43]:
elastic_net_grid_search.best_params_

{'alpha': 0.1, 'l1_ratio': 0.9}

In [44]:
elastic_net = ElasticNet(**elastic_net_grid_search.best_params_)
elastic_net.fit(X_train_norm, y_train)

summary(elastic_net, y_test, y_train, X_test_norm, X_train_norm)
elastic_net.coef_

На трейне R2: -9285.54767467615, MSE: 115094947200.11125
На тесте R2: -1352.8403190335966, MSE: 234682256769.28174


array([164124.11316674, -49214.35933319,  -1187.35878368,  25874.16544728,
       291618.94838964,  40062.10270451, -30906.38942728, -37089.43712877])

### **Задание 18 (0.35 балла)**

И так, вы обучили все варианты регуляризаций, доступные в sklearn, но не все, что мы обсудили на занятии! И следующее задание о забытой - $L0$ регуляризации.


**Ваша задача:**>
- [ ] Реализуйте модель с $L0$-регуляризаицей.
- [ ] Обучите модель на данных и проанализируйте результат, экспериментируя с коэффициенитом регуляризации
- [ ] Проанализируйте результаты обучения

**Важно:**

Вопреки тому, что L0 не реализована в стандартных пакетах, концепция данной регуляризации не является мертвой. Она может встречаться в [статьях](https://arxiv.org/abs/1712.01312) и экспериментах.



In [45]:
# your code here

# **Часть 3 (0.5 балла) | Добавляем категориальные фичи**

Попробуем для улучшения модели дать ей больше признаков. Добавим категориальные фичи.

За эту часть можно набрать 0.5 основных балла.



### **Задание 19 (0.1 балла)**

Проанализируйте столбец `name`. Очевидно, что эта переменная является категориальной, однако категорий в ней много.

- [ ] Предобработайте столбец `name`, чтобы избежать его удаления

In [46]:
X_train_cat = df_train.select_dtypes(exclude='number')
X_test_cat = df_test.select_dtypes(exclude='number')

In [47]:
X_train_cat['name'] = X_train_cat['name'].str.split(' ').str[0]
X_test_cat['name'] = X_test_cat['name'].str.split(' ').str[0] # Пойдем простым путем, можно было бы еще например выделить объем двигателя

In [48]:
# assert X_train_cat.shape == (5840, 11) # Не очень понимаю откуда может быть столько (11) категориальных фичей

In [49]:
X_train_cat.describe(include='object')

,name,fuel,seller_type,transmission,owner
count,5840,5840,5840,5840,5840
unique,30,4,3,2,5
top,Maruti,Diesel,Individual,Manual,First Owner
freq,1804,3177,5223,5336,3603


### **Задание 20 (0.1 балла)**

- [ ] Закодируйте категориалльные фичи и ``seats`` методом OneHot-кодирования.

In [50]:
X_train_cat = pd.merge(X_train_cat, df_train[['seats']], left_index=True, right_index=True)
X_test_cat = pd.merge(X_test_cat, df_test[['seats']], left_index=True, right_index=True)
X_train_cat['seats'] = X_train_cat['seats'].astype('category')
X_test_cat['seats'] = X_test_cat['seats'].astype('category')

In [51]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', drop='first')
enc.fit(X_train_cat)
X_train_cat_encod = enc.transform(X_train_cat).toarray()
X_test_cat_encod = enc.transform(X_test_cat).toarray()
enc.get_feature_names_out()

C:\Users\vyacheslav\anaconda3\envs\job1\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


array(['name_Audi', 'name_BMW', 'name_Chevrolet', 'name_Daewoo',
       'name_Datsun', 'name_Fiat', 'name_Force', 'name_Ford',
       'name_Honda', 'name_Hyundai', 'name_Isuzu', 'name_Jaguar',
       'name_Jeep', 'name_Kia', 'name_Land', 'name_Lexus', 'name_MG',
       'name_Mahindra', 'name_Maruti', 'name_Mercedes-Benz',
       'name_Mitsubishi', 'name_Nissan', 'name_Peugeot', 'name_Renault',
       'name_Skoda', 'name_Tata', 'name_Toyota', 'name_Volkswagen',
       'name_Volvo', 'fuel_Diesel', 'fuel_LPG', 'fuel_Petrol',
       'seller_type_Individual', 'seller_type_Trustmark Dealer',
       'transmission_Manual', 'owner_Fourth & Above Owner',
       'owner_Second Owner', 'owner_Test Drive Car', 'owner_Third Owner',
       'seats_4', 'seats_5', 'seats_6', 'seats_7', 'seats_8', 'seats_9',
       'seats_10', 'seats_14'], dtype=object)

### **Задание 21 (0.2 балла)**

OHE — базовый алгоритм преобразования категориальных признаков, но и с ним нужно быть аккуратными.

**Ответьте на вопросы:**


* Как корректно работать с OHE преобразованием?
* Почему мы удаляем один столбец?
* Пусть из $n$ признаков мы получили $n-1$ столбец, из которых $k < n -1$ оказались не важными по весам модели. Корректно ли их удалить?

1. Применять только к категориальным признакам. Не использовать для признаков с большим кол-вом уник значений. При необходимости удалять 1 из столбцов признака после преобразования OHE. При маленьком кол-ве наблюдений в каких либо фичах, объединить их в одну группу. Однинаковое кодирование на тесте и трейне.
2. Для борьбы с мультиколлинеарностью. Потому что без этого матрица X становится необратимой, что приведет к ошибке в расчете коэффициентов.
3. Да, можем удалить, но нужно проверить не приводит ли это к каким-либо проблемам. (можно просто применить регуляризацию).

### **Задание 22 (0.1 балла)**
Повторим то, что делали на прошлом шаге для моделей на вещественных признаках, однако теперь с моделью `Ridge`.


**Ваша задача:**
- [ ] Переберите параметр регуляризации `alpha` для гребневой (ridge) регрессии с помощью класса `GridSearchCV` В качестве параметров при объявлении GridSearchCV кроме модели укажите метрику качества $R^2$. Кроссвалидируйтесь по 10-ти фолдам.
- [ ] Ответье на вопрос: Удалось ли улучшить качество прогнозов?

In [52]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [53]:
X_train_norm = np.delete(X_train_norm, 6, axis=1) # удалим seats
X_train_combined = np.hstack([X_train_cat_encod, X_train_norm])

X_test_norm = np.delete(X_test_norm, 6, axis=1) # удалим seats
X_test_combined = np.hstack([X_test_cat_encod, X_test_norm])

In [54]:
param_grid = {
    'alpha': [0.01, 0.1, 0.5, 1, 5]
}
ridge = Ridge()

grid_search_ridge = GridSearchCV(
    estimator=ridge,
    param_grid=param_grid,
    scoring='r2',
    cv=10,
    verbose=1,
    n_jobs=-1
)

grid_search_ridge.fit(X_train_combined, y_train)

summary(grid_search_ridge.best_estimator_, y_test, y_train, X_test_combined, X_train_combined)

Fitting 10 folds for each of 5 candidates, totalling 50 fits
На трейне R2: 0.7733768834222516, MSE: 64958873569.33142
На тесте R2: 0.7823721370603648, MSE: 125098710120.27187


Качество удалось улучшить, однако наблюдаем проблемы с переобучением. Следовательно эту модель мы не можем оставить как финальную

# **Часть 4 - бонусная (1 балл) | Feature Engineering**

В этой части домашнего задания вам предлагается проявить свою креативность для улучшения прогноза модели. Любые другие модели, кроме различных форм линейной (или полиномиальной) регресси, использовать запрещается. А значит, придется работать с признаками

**Что можно попробовать сделать?** (каждый пункт по 0.4 балла, но не больше 2-х баллов в сумме)

1.   *Сгенерировать новые признаки на основе уже существующих:*
    * посчитать произведения // частные признаков (кажется, что посчитать число "лошадей" на литр объема может быть полезно);
    * имеет смысл обратить внимание на визуализации в части с EDA (к примеру, зависимость цены от года выглядит квадратичной, а не линейной; значит, квадрат года нам, скорее всего, принесет больше пользы)

2.   *Добыть новые признаки:*
    * имеем название автомобиля, которое никак не используем (можно спарсить инфу о классе автомобиля или каких-то специфических опциях)
    * можно добавить пороговые признаки вроде "владелец третий или больше" и объединить признаки в некоторые осмысленные правила, например "первый или второй владелец и продавец официальный дилер" (подбирать пороги удобно по диаграммам рассеяния)

3.   *Поработать с уже имеющимися:*
    * далеко не факт, что заполнить пропуск медианой было лучшей идеей (как минимум, можно добавить dummy-столбец для модели, сигнализирующий, что раньше на месте медианы был пропуск -- там где он был, конечно); попробуйте другие способы филлинга;
    * мы не анализировали, есть ли в данных выбросы => никак выбросы не обрабатывали; наиболее простым и, тем не менее, довольно полезным вариантом нахождения выбросов могут послужить boxplot'ы для каждого столбца; что делать с выбросами думайте сами :) -- вариантов довольно много
    * мы толком не смотрели на таргет сам по себе; в нем тоже могут быть неожиданности -- стоит хотя бы проверить
    * можно заметить, что некоторые признаки распределены совсем не нормально; возможно их стоит отлогарифмировать

И так далее...

Feel Free to Try!

In [55]:
df['year'].max()

2020

In [56]:
X_train_pipeline = pd.merge(X_train_cat, X_train.drop(columns=['seats']), left_index=True, right_index=True)
X_test_pipeline = pd.merge(X_test_cat, X_test.drop(columns=['seats']), left_index=True, right_index=True)

In [57]:
X_train_pipeline['average_km_per_year'] = X_train_pipeline['km_driven'] / (2021 - X_train_pipeline['year']) # Максимальный год 2020, следовательно предположение, что это год создания ДС
X_train_pipeline['hp_per_liter'] = X_train_pipeline['max_power'] / (X_train_pipeline['engine'] / 1000)

X_test_pipeline['average_km_per_year'] = X_test_pipeline['km_driven'] / (2021 - X_test_pipeline['year']) # Максимальный год 2020, следовательно предположение, что это год создания ДС
X_test_pipeline['hp_per_liter'] = X_test_pipeline['max_power'] / (X_test_pipeline['engine'] / 1000)

In [58]:
X_train_pipeline.select_dtypes(exclude='number').columns

Index(['name', 'fuel', 'seller_type', 'transmission', 'owner', 'seats'], dtype='object')

In [59]:
X_train_pipeline.select_dtypes(include='number').columns

Index(['year', 'km_driven', 'mileage', 'engine', 'max_power', 'torque',
       'max_torque_rpm', 'average_km_per_year', 'hp_per_liter'],
      dtype='object')

In [69]:
X_train_pipeline.iloc[0, :].to_dict()

{'name': 'Maruti',
 'fuel': 'Diesel',
 'seller_type': 'Individual',
 'transmission': 'Manual',
 'owner': 'First Owner',
 'seats': 5,
 'year': 2014,
 'km_driven': 145500,
 'mileage': 23.4,
 'engine': 1248,
 'max_power': 74.0,
 'torque': 190.0,
 'max_torque_rpm': 2000.0,
 'average_km_per_year': 20785.714285714286,
 'hp_per_liter': 59.294871794871796}

In [306]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer


numerical_features = X_train_pipeline.select_dtypes(include='number').columns
categorical_features = X_train_pipeline.select_dtypes(exclude='number').columns

num_transformers = [
    ('year', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('poly', PolynomialFeatures(degree=1, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['year']),
    ('km_driven', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('poly', PolynomialFeatures(degree=1, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['km_driven']),
    ('mileage', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('poly', PolynomialFeatures(degree=1, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['mileage']),
    ('engine', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('poly', PolynomialFeatures(degree=1, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['engine']),
    ('max_power', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('poly', PolynomialFeatures(degree=1, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['max_power']),
    ('torque', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('poly', PolynomialFeatures(degree=1, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['torque']),
    ('max_torque_rpm', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('poly', PolynomialFeatures(degree=1, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['max_torque_rpm']),
    ('average_km_per_year', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('poly', PolynomialFeatures(degree=1, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['average_km_per_year']),
    ('hp_per_liter', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('poly', PolynomialFeatures(degree=1, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['hp_per_liter']),
]

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='NA')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

preprocessor = ColumnTransformer(
    transformers=num_transformers + [
        ('cat', categorical_transformer, categorical_features)
    ]
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge())
])

param_grid = {
    # 'preprocessor__mileage__imputer__strategy': ['mean', 'median'],
    # 'preprocessor__engine__imputer__strategy': ['mean', 'median'],
    # 'preprocessor__max_power__imputer__strategy': ['mean', 'median'],
    # 'preprocessor__torque__imputer__strategy': ['mean', 'median'],
    # 'preprocessor__seats__imputer__strategy': ['mean', 'median'],
    # 'preprocessor__max_torque_rpm__imputer__strategy': ['mean', 'median'],

    'preprocessor__year__poly__degree': [1, 2],
    'preprocessor__km_driven__poly__degree': [1, 2],
    'preprocessor__mileage__poly__degree': [1, 2],
    'preprocessor__engine__poly__degree': [1, 2],
    'preprocessor__max_power__poly__degree': [1, 2],
    'preprocessor__torque__poly__degree': [1, 2],
    'preprocessor__max_torque_rpm__poly__degree': [1, 2],
    'preprocessor__average_km_per_year__poly__degree': [1, 2],
    'preprocessor__hp_per_liter__poly__degree': [1, 2],
    'regressor__alpha': [0.1, 1.0, 10.0]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train_pipeline, y_train)

Fitting 5 folds for each of 1536 candidates, totalling 7680 fits


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
 

Лучшие параметры: {'preprocessor__average_km_per_year__poly__degree': 1, 'preprocessor__engine__poly__degree': 1, 'preprocessor__hp_per_liter__poly__degree': 2, 'preprocessor__km_driven__poly__degree': 2, 'preprocessor__max_power__poly__degree': 2, 'preprocessor__max_torque_rpm__poly__degree': 2, 'preprocessor__mileage__poly__degree': 1, 'preprocessor__torque__poly__degree': 2, 'preprocessor__year__poly__degree': 2, 'regressor__alpha': 1.0}
Лучшее значение MSE: 70968091353.51584
MSE на тестовых данных: 113647868900.50375


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [307]:
summary(grid_search.best_estimator_, y_test, y_train, X_test_pipeline, X_train_pipeline)

На трейне R2: 0.8028753128585142, MSE: 56503492771.55371
На тесте R2: 0.8022925830915297, MSE: 113647868900.50375


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [61]:
numerical_features = X_train_pipeline.select_dtypes(include='number').columns
categorical_features = X_train_pipeline.select_dtypes(exclude='number').columns


num_transformers = [
    ('year', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['year']),
    ('km_driven', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['km_driven']),
    ('mileage', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('poly', PolynomialFeatures(degree=1, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['mileage']),
    ('engine', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('poly', PolynomialFeatures(degree=1, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['engine']),
    ('max_power', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['max_power']),
    ('torque', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['torque']),
    ('max_torque_rpm', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['max_torque_rpm']),
    ('average_km_per_year', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('poly', PolynomialFeatures(degree=1, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['average_km_per_year']),
    ('hp_per_liter', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
        ('scaler', StandardScaler())
    ]), ['hp_per_liter']),
]

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='NA')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

preprocessor = ColumnTransformer(
    transformers=num_transformers + [
        ('cat', categorical_transformer, categorical_features)
    ]
)

regressor = Ridge(alpha=1.0)

final_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', regressor)
])


model_with_mean = final_pipeline.fit(X_train_pipeline, y_train)

summary(model_with_mean, y_test, y_train, X_test_pipeline, X_train_pipeline)

На трейне R2: 0.8028753128584881, MSE: 56503492771.56122
На тесте R2: 0.802292583091533, MSE: 113647868900.50186


C:\Users\vyacheslav\anaconda3\envs\job1\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [71]:
import pickle

    
with open('car_price_model.pkl', 'wb') as model_file:
    pickle.dump(model_with_mean, model_file)

# **Часть 4. | Бизнесовая (0.5 балла)**

### **Задание 23 (0.25 балла)**

В мире бизнеса очень важно давать оценку качества модели понятную бизнесу, поэтому иногда заказчики приходят с кастомными метриками. Попробуем сделать такую для нашей задачи.

**Описание метрики:**

Среди всех предсказанных цен на авто нужно посчитать долю прогнозов, отличающихся от реальных цен на эти авто не более чем на 10% (в одну или другую сторону)

**Ваша задача:**

- [ ] Реализуйте метрику `business_metric`
- [ ] Посчитайте метрику для всех обученных моделей и определеите, какаю лучше всего решает задачу бизнеса

In [60]:
def business_metrics(y_true, y_pred):
    result = np.sum(np.where(np.abs(y_true - y_pred) / y_true <= 0.1, 1, 0)) / len(y_true)
    return result
        

business_metrics(y_test, grid_search_ridge.best_estimator_.predict(X_test_combined)) # Модель с комбинированными признаками

0.307

In [65]:
business_metrics(y_test, model.predict(X_test)) # Модель только на числовых 

C:\Users\vyacheslav\anaconda3\envs\job1\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


0.0

In [79]:
business_metrics(y_test, grid_search.best_estimator_.predict(X_test_pipeline)) # Лучшая итоговая модель

C:\Users\vyacheslav\anaconda3\envs\job1\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


0.277

### **Задание 24 (0.25 балла)**

Но у бизнеса не всегда есть идеи и иногда задача на выбор метрики делегируется вам.

**Задание:**

- [ ] Придумайте и реализуйте другую кастомную метрику с учетом того, что модель не должна сильно ошибаться в прогнозе, но недопрогноз для модели, согласно мнению бизнеса, хуже, чем перепрогноз.
- [ ] Посчитайте метрику для всех обученных моделей и определеите, какаю лучше всего решает задачу бизнеса



# **Часть 5 (3 балла) | Реализация сервиса на FastAPI**

### **Задание 25**

Cделайте с помощью FastAPI сервис, который с точки зрения пользователя реализует две функции:

1. на вход в формате json подаются признаки одного объекта, на выходе сервис выдает предсказанную стоимость машины
2. на вход подается csv-файл с признаками тестовых объектов, на выходе получаем файл с +1 столбцом - предсказаниями на этих объектах

С точки зрения реализации это означает следующее:
- средствами pydantic должен быть описан класс базового объекта
- класс с коллецией объектов
- метод post, который получает на вход один объект описанного класса
- метод post, который получает на вход коллекцию объектов описанного класса

Шаблон для сервисной части дан ниже. Код необходимо дополнить и оформить в виде отдельного .py-файла.

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List

app = FastAPI()


class Item(BaseModel):
    name: str
    year: int
    selling_price: int
    km_driven: int
    fuel: str
    seller_type: str
    transmission: str
    owner: str
    mileage: str
    engine: str
    max_power: str
    torque: str
    seats: float


class Items(BaseModel):
    objects: List[Item]


@app.post("/predict_item")
def predict_item(item: Item) -> float:
    return ...


@app.post("/predict_items")
def predict_items(items: List[Item]) -> List[float]:
    return ...

Протестируйте сервис на корректность работы и приложите скриншоты (см. ниже).

# **Часть 6 (1 балл) | Оформление результатов**

### **Задание 26**

**Результаты вашей работы** необходимо разместить в своем Гитхабе. Под результатами понимаем следующее:
* ``.ipynb``-ноутбук со всеми проведёнными вами экспериментами (output'ы ячеек, разумеется, сохранить)
* ``.py``-файл с реализацией сервиса
* ``.pickle``-файл с сохранёнными весами модели, коэффициентами скейлинга и прочими числовыми значениями, которые могут понадобиться для инференса
* ``.md``-файл с выводами про проделанной вами работе:
    * что было сделано
    * с какими результатами
    * что дало наибольший буст в качестве
    * что сделать не вышло и почему (это нормально, даже хорошо😀)

**За что могут быть сняты баллы в этом пункте:**
* за отсутствие ``.pickle``-файла с весами использованной модели
* за недостаточную аналитику в ``.md``-файле
* за оформление и логику кода (в определённом смысле это тоже элемент оформления решения)

**Как будет выглядет проверка всего домашнего задания?**
1. Ассистент проходит по ссылке на (**открытый**) репозиторий из Энитаска
2. Смотрит ``readme.md``:
    * пожалуйста, приложите в него же скрины работы вашего сервиса -- собирать ваши проекты довольно времязатратно, но хочется убедиться, что всё работает
    * можете в md-файл приложить ссылку на screencast с демонстрацией

3. Просматривает ноутбук с DS частью
4. Заглядывает в код сервиса
5. Хвалит

# **Часть Благодарственная**

Надеемся, вы честно проделали все пункты, а не просто пролистали досюда. Потому что здесь награда за старания. Пожалуйста, не стоит награждать себя до того, как закончите работать над домашкой.

<details>
<summary><b>Что-то приятное</b></summary>

**Напоминаем, что нашем курсе действует система кото-бонусов** 🐈

На фото по ссылке — сэр кот кого-то из команды курса (преподаватель, помощник преподавателя, ассистенты).

Предлагаем вам угадать — чей это товарищ!

[Первый кот](https://ibb.co/Kz1ZRWj)

</details>